# Tarea 9
## Dask
### Federico Riveroll

In [58]:
from dask.distributed import Client
from dask import bag
import json
from dask import delayed
from dask import dataframe
import numpy as np 
import pandas as pd
import time
from dask import delayed
from datetime import datetime, timedelta
from sklearn.cross_validation import train_test_split 
from sklearn.preprocessing import StandardScaler,LabelEncoder,FunctionTransformer,OneHotEncoder
from sklearn.pipeline import make_pipeline 
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import make_pipeline 
from sklearn.pipeline import Pipeline
client = Client("scheduler:8786")

In [66]:
# Lectura y preparación de variables
trips_df = dataframe.read_csv("/data/trips.csv")

trips_df.tpep_pickup_datetime = trips_df.tpep_pickup_datetime.astype('M8[us]')
trips_df.tpep_dropoff_datetime = trips_df.tpep_dropoff_datetime.astype('M8[us]')
trips_df["duration"]=trips_df["tpep_dropoff_datetime"]-trips_df["tpep_pickup_datetime"]
trips_df["proportion"]=trips_df["tip_amount"]/trips_df["fare_amount"] 
trips_df["r_duration_distance"] = (trips_df["duration"]/ timedelta (days=1)) / (trips_df["trip_distance"] + 0.0001)
trips_var=trips_df[["car_type","passenger_count","trip_distance","duration", "r_duration_distance", "proportion"]]
trips_var.duration=trips_var.duration/ timedelta (days=1) 
trips_var=trips_var.fillna(trips_var.mean()).compute()


df1 = trips_var
df1.head()

,car_type,passenger_count,trip_distance,duration,r_duration_distance,proportion
0,A,1,6.90,0.013542,0.001963,0.209091
1,A,1,1.81,0.006794,0.003753,0.000000
2,A,1,0.96,0.007257,0.007559,0.133333
3,A,1,1.90,0.006377,0.003356,0.117647
4,A,1,1.00,0.006574,0.006573,0.221333


In [67]:
# Pre procesamiento (sólo pasé a one-hot la variable car_type)
mlb = MultiLabelBinarizer()
campos_a_onehot = ['car_type']

for columna in campos_a_onehot:
    df1 = df1.join(pd.DataFrame(mlb.fit_transform(df1.pop(columna).apply(str)),
                              index=df1.index,
                              columns=[(mlb.classes_[i] + "_" + columna) for i in range(len(mlb.classes_))]))

df1.head()

,passenger_count,trip_distance,duration,r_duration_distance,proportion,A_car_type,B_car_type
0,1,6.90,0.013542,0.001963,0.209091,1,0
1,1,1.81,0.006794,0.003753,0.000000,1,0
2,1,0.96,0.007257,0.007559,0.133333,1,0
3,1,1.90,0.006377,0.003356,0.117647,1,0
4,1,1.00,0.006574,0.006573,0.221333,1,0


In [68]:
# Separación
X=np.array(df1[["A_car_type","B_car_type","passenger_count","trip_distance","duration", "r_duration_distance"]])
y=np.array(df1['proportion']) 
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=1)

In [69]:
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(max_depth=25) 
regressor.fit(X_train, y_train) 

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=25,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

### Secuencial

In [70]:
%%time
parameters = {'max_depth':[10, 20, 30, 40, 50]}
grid = GridSearchCV(regressor,parameters, cv=10)
grid.fit(X_train, y_train)
print("Best Score : ", grid.best_score_)

r2 / variance :  -0.03440429756955328
CPU times: user 11.8 s, sys: 58.3 ms, total: 11.8 s
Wall time: 11.7 s


### Paralelo

In [71]:
%%time
parameters = {'max_depth':[10, 20, 30, 40, 50]}
grid = delayed(GridSearchCV(regressor, parameters, cv=10))
grid.fit(X_train, y_train)
print("Best Score : ", grid.best_score_)

Best Score :  Delayed('getattr-f6ea5679bd070aa47f96c4e7ec41276c')
CPU times: user 3.4 ms, sys: 0 ns, total: 3.4 ms
Wall time: 2.68 ms


En paralelo es mucho más rápido, es bastante impresionante este truco.